### equality constraints

    x10 + x20 + x30 = 1224
    x11 + x21 + x31 = 216
    x12 + x22 + x32 = 216
    x13 + x23 + x33 = 144
    x14 + x24 + x34 = 480
    x15 + x25 + x35 = 120

    x10 + x11 + x12 + x13 + x14 + x15 = 1800
    x20 + x21 + x22 + x23 + x24 + x25 + x30 + x31 + x32 + x33 + x34 + x35 = 600
    
    x20 + x21 + x22 + x23 + x24 + x25 - p1*600 = 0
    x30 + x31 + x32 + x33 + x34 + x35 - p2*600 = 0
    
    p1 + p2 = 1
    

### inequality constraints

    None

### bounds

    ok, default = (0,None)
    p1,p2 binary
    
### objective function

In [1]:
from pulp import *

In [2]:
variables = ['x'+''.join(map(str,divmod(x,6))) for x in range(18)]
plants = ['p'+str(x) for x in range(2)]

LP_variables = [LpVariable(x, lowBound = 0, cat='Integer') for x in variables]
LP_variables += [LpVariable(x, lowBound = 0, cat='Binary') for x in plants]
print(LP_variables)

[x00, x01, x02, x03, x04, x05, x10, x11, x12, x13, x14, x15, x20, x21, x22, x23, x24, x25, p0, p1]


In [3]:
# consumption per warehouse
LP_expressions = [LpAffineExpression([(LP_variables[x],1),
                                      (LP_variables[x+6],1),
                                      (LP_variables[x+12],1)]) for x in range(6)]
# plants
LP_expressions += [LpAffineExpression([(LP_variables[-2],1),
                                        (LP_variables[-1],1)])]

# production per plant
LP_expressions += [LpAffineExpression([(LP_variables[x],1) for x in range(6)])]
LP_expressions += [LpAffineExpression([(LP_variables[x],1) for x in range(6,18)])]

LP_expressions += [LpAffineExpression([(LP_variables[x+y*6],1) for x in range(6)]
                                      +[(LP_variables[y-3],-600)]
                                     ) for y in range(1,3)]

In [4]:
rhs = [1224,216,216,144,480,120,1,1800,600,0,0]

In [5]:
LP_constraints = [LpConstraint(x,sense=0,rhs=rhs[ix]) for ix,x in enumerate(LP_expressions)]

In [6]:
# objective function
costs = [0.8,1.2,1.5,1.6,1.9,2.1, 1.3,1.9,1.05,0.8,1.5,1.7, 1.9,2.9,1.2,1.6,0.9,0.8]

LP_function = LpAffineExpression([(x,y) for x,y in zip(LP_variables,costs)])

In [7]:
prob = LpProblem("The_Plant_Problem",LpMinimize)
prob += LP_function

for x in LP_constraints:
    prob += x
prob


The_Plant_Problem:
MINIMIZE
0.8*x00 + 1.2*x01 + 1.5*x02 + 1.6*x03 + 1.9*x04 + 2.1*x05 + 1.3*x10 + 1.9*x11 + 1.05*x12 + 0.8*x13 + 1.5*x14 + 1.7*x15 + 1.9*x20 + 2.9*x21 + 1.2*x22 + 1.6*x23 + 0.9*x24 + 0.8*x25 + 0
SUBJECT TO
_C1: x00 + x10 + x20 = 1224

_C2: x01 + x11 + x21 = 216

_C3: x02 + x12 + x22 = 216

_C4: x03 + x13 + x23 = 144

_C5: x04 + x14 + x24 = 480

_C6: x05 + x15 + x25 = 120

_C7: p0 + p1 = 1

_C8: x00 + x01 + x02 + x03 + x04 + x05 = 1800

_C9: x10 + x11 + x12 + x13 + x14 + x15 + x20 + x21 + x22 + x23 + x24 + x25
 = 600

_C10: - 600 p0 + x10 + x11 + x12 + x13 + x14 + x15 = 0

_C11: - 600 p1 + x20 + x21 + x22 + x23 + x24 + x25 = 0

VARIABLES
0 <= p0 <= 1 Integer
0 <= p1 <= 1 Integer
0 <= x00 Integer
0 <= x01 Integer
0 <= x02 Integer
0 <= x03 Integer
0 <= x04 Integer
0 <= x05 Integer
0 <= x10 Integer
0 <= x11 Integer
0 <= x12 Integer
0 <= x13 Integer
0 <= x14 Integer
0 <= x15 Integer
0 <= x20 Integer
0 <= x21 Integer
0 <= x22 Integer
0 <= x23 Integer
0 <= x24 Integer
0 <= x25

In [8]:
prob.writeLP("The_Plant_Problem.lp")
prob.solve()

1

In [9]:
for v in prob.variables():
    if v.varValue:
        print(v.name, "=", int(v.varValue))
        
print("Total Cost = ", value(prob.objective))

p1 = 1
x00 = 1224
x01 = 216
x02 = 216
x03 = 144
x24 = 480
x25 = 120
Total Cost =  2320.8


### equality constraints
    
    center + south = 1

    demand is met in all warehouses
    due to float, demand is floor of float
    

### inequality constraints

    supply >= demand
    
### bounds

    ok, default = (0,None)
    p1,p2 binary
    
### objective function

    cost * (supply_per_plant ^ warehouse ^ good)

In [10]:
import numpy as np
from pulp import *

# sum demand over years
Demand = np.array([2400+2450+2600,1750+2000+2300,490+550+600])
Plant1 = np.array([1800+1400+1900,1250+1350+1400,350+380+400])

Plant2 = Demand - Plant1

Zone = np.array([[0.6],[0.15],[0.25]])
House = np.array([[0.85,0.15],[0.6,0.4],[0.8,0.2]])
Warehouse = (House * Zone).reshape(-1,1)

# floor demand
Local_demand = (Warehouse * Demand).astype(int)

Cost = np.array([[0.8,1.3,1.9],[1.2,1.9,2.9],[1.5,1.05,1.2],
                 [1.6,0.8,1.6],[1.9,1.5,0.9],[2.1,1.7,0.8]])

In [11]:
Local_demand

array([[3799, 3085,  836],
       [ 670,  544,  147],
       [ 670,  544,  147],
       [ 447,  363,   98],
       [1490, 1210,  328],
       [ 372,  302,   82]])

In [12]:
variables = ['w_'+str(x).zfill(2) for x in range(1,18*3+1)]
plants = ['p_'+str(x) for x in range(2)]

LP_variables = [LpVariable(x, lowBound = 0, cat='Integer') for x in variables]
LP_variables += [LpVariable(x, lowBound = 0, cat='Binary') for x in plants]

In [13]:
# plants
LP_expressions = [LpAffineExpression([(LP_variables[-2],1),
                                        (LP_variables[-1],1)])]

# consumption per warehouse
for y in range(3):
    LP_expressions += [LpAffineExpression([(LP_variables[18*y+x],1),
                                      (LP_variables[18*y+x+6],1),
                                      (LP_variables[18*y+x+12],1)]) for x in range(6)]

# supply per plant
for y in range(3):
    LP_expressions += [LpAffineExpression([(LP_variables[x],1) for x in range(18*y,18*y+6)])]
for y in range(3):
    LP_expressions += [LpAffineExpression([(LP_variables[x+z*6],1) for x in range(18*y,18*y+6)]
                                          +[(LP_variables[z-3],-Plant2[y])]
                                                     ) for z in range(1,3)]
for y in range(3):
    LP_expressions += [LpAffineExpression([(LP_variables[x],1) for x in range(18*y+6,18*y+18)])]


In [14]:
rhs_equal = [1] + np.transpose(Local_demand).reshape(-1,).tolist() 
rhs_small = Plant1.tolist()  + [0]*6 + Plant2.tolist()

LP_constraints = [LpConstraint(x,sense=0,rhs=rhs_equal[ix]) for ix,x in enumerate(LP_expressions[:-12])]
LP_constraints += [LpConstraint(x,sense=-1,rhs=rhs_small[ix]) for ix,x in enumerate(LP_expressions[-12:])]

In [15]:
costs = [0.8,1.2,1.5,1.6,1.9,2.1, 1.3,1.9,1.05,0.8,1.5,1.7, 1.9,2.9,1.2,1.6,0.9,0.8]

LP_function = LpAffineExpression([(x,y) for x,y in zip(LP_variables,costs*3)])

In [16]:
prob = LpProblem("The_Plant_Problem",LpMinimize)
prob += LP_function

for x in LP_constraints:
    prob += x
prob

The_Plant_Problem:
MINIMIZE
0.8*w_01 + 1.2*w_02 + 1.5*w_03 + 1.6*w_04 + 1.9*w_05 + 2.1*w_06 + 1.3*w_07 + 1.9*w_08 + 1.05*w_09 + 0.8*w_10 + 1.5*w_11 + 1.7*w_12 + 1.9*w_13 + 2.9*w_14 + 1.2*w_15 + 1.6*w_16 + 0.9*w_17 + 0.8*w_18 + 0.8*w_19 + 1.2*w_20 + 1.5*w_21 + 1.6*w_22 + 1.9*w_23 + 2.1*w_24 + 1.3*w_25 + 1.9*w_26 + 1.05*w_27 + 0.8*w_28 + 1.5*w_29 + 1.7*w_30 + 1.9*w_31 + 2.9*w_32 + 1.2*w_33 + 1.6*w_34 + 0.9*w_35 + 0.8*w_36 + 0.8*w_37 + 1.2*w_38 + 1.5*w_39 + 1.6*w_40 + 1.9*w_41 + 2.1*w_42 + 1.3*w_43 + 1.9*w_44 + 1.05*w_45 + 0.8*w_46 + 1.5*w_47 + 1.7*w_48 + 1.9*w_49 + 2.9*w_50 + 1.2*w_51 + 1.6*w_52 + 0.9*w_53 + 0.8*w_54 + 0
SUBJECT TO
_C1: p_0 + p_1 = 1

_C2: w_01 + w_07 + w_13 = 3799

_C3: w_02 + w_08 + w_14 = 670

_C4: w_03 + w_09 + w_15 = 670

_C5: w_04 + w_10 + w_16 = 447

_C6: w_05 + w_11 + w_17 = 1490

_C7: w_06 + w_12 + w_18 = 372

_C8: w_19 + w_25 + w_31 = 3085

_C9: w_20 + w_26 + w_32 = 544

_C10: w_21 + w_27 + w_33 = 544

_C11: w_22 + w_28 + w_34 = 363

_C12: w_23 + w_29 + w_35 = 

In [17]:
prob.writeLP("The_Plant_Problem.lp")
prob.solve()

1

In [18]:
for v in prob.variables():
    if v.varValue:
        print(v.name, "=", int(v.varValue))
        
print("Total Cost = ", value(prob.objective))

p_1 = 1
w_01 = 3799
w_02 = 670
w_03 = 182
w_04 = 447
w_15 = 488
w_17 = 1490
w_18 = 372
w_19 = 3085
w_20 = 544
w_21 = 6
w_22 = 363
w_33 = 538
w_35 = 1210
w_36 = 302
w_37 = 836
w_38 = 147
w_39 = 47
w_40 = 98
w_51 = 100
w_53 = 328
w_54 = 82
Total Cost =  14295.7
